In [2]:
!pip install scikit-optimize
!pip install scikit-learn
!pip install datasets transformers pandas numpy

from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Load the dataset
df = pd.read_csv("FinancES_train_kaggle.csv", encoding="utf-8")
print("Dataset shape:", df.shape)

# Split into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load BETO model and tokenizer
model_checkpoint = "UMUTeam/roberta-spanish-sentiment-analysis"
num_labels = len(df["label"].unique())
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["id", "text"])
val_dataset = val_dataset.remove_columns(["id", "text"])

# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"f1_macro": f1_score(labels, preds, average="macro")}

# Define the search space
space = [
    Real(1e-6, 1e-4, "log-uniform", name="learning_rate"),
    Categorical([8, 16, 32], name="batch_size"),
    Integer(3, 10, name="num_epochs"),
    Real(1e-4, 1e-2, "log-uniform", name="weight_decay"),
    Categorical([1, 2, 4], name="gradient_accumulation_steps")
]

# Objective function for Bayesian optimization
@use_named_args(space)
def objective(**params):
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

    training_args = TrainingArguments(
        output_dir="beto-checkpoints",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=params["learning_rate"],
        per_device_train_batch_size=params["batch_size"],
        num_train_epochs=params["num_epochs"],
        weight_decay=params["weight_decay"],
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        fp16=True,
        gradient_accumulation_steps=params["gradient_accumulation_steps"],
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate()
    return -metrics["eval_f1_macro"]  # Negative because we want to maximize

# Run Bayesian optimization
result = gp_minimize(objective, space, n_calls=10, random_state=42)

# Print best parameters and score
print("Best parameters:")
for name, value in zip(["learning_rate", "batch_size", "num_epochs", "weight_decay", "gradient_accumulation_steps"], result.x):
    print(f"  {name}: {value}")
print(f"Best F1 Macro: {-result.fun}")

# Train final model with best hyperparameters
best_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
best_training_args = TrainingArguments(
    output_dir="beto-checkpoints-best",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=result.x[0],
    per_device_train_batch_size=result.x[1],
    num_train_epochs=result.x[2],
    weight_decay=result.x[3],
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True,
    gradient_accumulation_steps=result.x[4],
    report_to="none"
)

best_trainer = Trainer(
    model=best_model,
    args=best_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

best_trainer.train()

# Final evaluation and prediction generation
final_metrics = best_trainer.evaluate()
print("F1 Macro final (validación):", final_metrics["eval_f1_macro"])

test_df = pd.read_csv("FinancES_test_kaggle.csv", encoding="utf-8")
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.remove_columns(["id", "text"])

predictions = best_trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

submission = pd.DataFrame({"id": test_df["id"], "label": preds})
submission.to_csv("submission_roberta_optimized.csv", index=False)
print("Archivo 'submission_roberta_optimized.csv' creado con las predicciones optimizadas.")


  Using cached scikit_optimize-0.10.2-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached pyaml-25.1.0-py3-none-any.whl.metadata (12 kB)
Using cached scikit_optimize-0.10.2-py2.py3-none-any.whl (107 kB)
Using cached pyaml-25.1.0-py3-none-any.whl (26 kB)
Dataset shape: (6359, 3)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-0875c8345480>:116: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  best_trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1 Macro
1,No log,0.524782,0.719771
2,No log,0.518967,0.744304
3,No log,0.616316,0.753566


F1 Macro final (validación): 0.7535662335410075


Map:   0%|          | 0/1621 [00:00<?, ? examples/s]

Archivo 'submission_roberta_optimized.csv' creado con las predicciones optimizadas.
